In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import re
from string import punctuation
from random import sample

In [2]:
df1 = pd.read_csv("../data/Duplicate_Bug_Report/Eclipse/EP_dup.csv", sep=";")
df2 = pd.read_csv("../data/Duplicate_Bug_Report/Eclipse/EP_nondup.csv", sep=";")
df3 = pd.read_csv("../data/Duplicate_Bug_Report/Mozilla/M_Duplicate BRs.csv", sep=";")  
df4 = pd.read_csv("../data/Duplicate_Bug_Report/Mozilla/M_NonDuplicate BRs.csv", sep=";")
df5 = pd.read_csv("../data/Duplicate_Bug_Report/ThunderBird/dup_TB.csv", sep=";")
df6 = pd.read_csv("../data/Duplicate_Bug_Report/ThunderBird/Nondup_TB.csv", sep=";")

frames=[df1,df2,df3,df4,df5,df6]
data=pd.concat(frames)
data

,Issue_id,Duplicated_issue,Title1,Description1,Title2,Description2,Label
0,25,28126,cvs ui need vcm prefs default repo connection gc,it would be helpful if there was a notion of d...,wizards patch standard public cvs repositories,this patch adds a convenient way to check thin...,1
1,40,20,need connect to team stream gcqpkw,i would like to be able to connect to a team s...,workspace files,thought it would be useful if the set of repo ...,1
2,48,22,make sure can future store other project refer...,project references come in three flavours . p...,persist sharing recommendations and project ve...,project descriptions dont store sharing recomm...,1
3,61,60,.vcmmeta showing as change gdqtgw,useruser install drop into declipse user ...,need custom .vcmignore comparemerge gdqt,useruser install drop into declipse user ...,1
4,94,2,repositories view all file types open to the t...,when browsing files in the repositories view i...,opening repository resources doesnt honor type...,opening repository resource open the default ...,1
...,...,...,...,...,...,...,...
9900,418747,953052,thunderbird outlook migrate lost outlooks pst ...,i installed thunderbird hoping i could impor...,i am using thunderbird esr latest but i keep ...,i am using thunderbird .. esr latest but i...,0
9901,418821,953165,urls in mail with comma opens with c in browser,when clicking in text mail not html on url w...,picture in feed not showing,. i linked the rssfeed from httpwww.nu.nlf...,0
9902,418832,953249,greek attachments not displayed correctly,i am using gmail imap accounts. when i recei...,add the flagcolpng to linux,httpmxr.mozilla.orgcommcentralsearchstringflag...,0
9903,419035,953375,branch only random crash when i want to displ...,the problem appears with icedove the debian ...,replace undefined gpromptservice with services...,patch v discovered a call to undefined gprom...,0


In [3]:
data = data.reset_index()

In [4]:
data['TitleDescription1'] = data['Title1'].str.cat(data['Description1'],sep=" ")
data['TitleDescription2'] = data['Title2'].str.cat(data['Description2'],sep=" ")

In [5]:
stop_words = set(stopwords.words('english'))

In [6]:
def words(text, remove_stop_words=True, stem_words=False):
    # Remove punctuation from questions
    text = ''.join([c for c in text if c not in punctuation])
    
    # Lowering the words in questions
    text = text.lower()
    
    # Remove stop words from questions
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Return a list of words
    return(text)

In [7]:
def process(bug_list, bugs):
    for bug in bugs:
        bug_list.append(words(bug))
processed_bug1 = []
processed_bug2 = []
process(processed_bug1, data.TitleDescription1)
process(processed_bug2, data.TitleDescription2)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer = 'word',
                        stop_words = 'english',
                        lowercase = True,
                        max_features = 300,
                        norm = 'l1')

In [9]:
words = pd.concat([data.TitleDescription1, data.TitleDescription2], axis = 0)

In [31]:
tfidf.fit(words)
print(tfidf.vocabulary_)
duplicate_1 = tfidf.transform(data.TitleDescription1)
duplicate_2 = tfidf.transform(data.TitleDescription2)

{'cvs': 58, 'ui': 270, 'need': 169, 'default': 61, 'project': 205, 'list': 146, 'sure': 246, 'created': 53, 'create': 52, 'type': 269, 'team': 252, 'like': 141, 'able': 0, 'add': 6, 'content': 43, 'workspace': 296, 'added': 7, 'new': 170, 'currently': 55, 'open': 177, 'browser': 23, 'select': 226, 'used': 274, 'dont': 72, 'think': 256, 'make': 156, 'version': 278, 'use': 273, 'time': 259, 'file': 94, 'example': 86, 'xml': 299, 'right': 217, 'different': 65, 'fine': 96, 'change': 30, 'install': 119, 'user': 275, 'java': 128, 'perspective': 191, 'class': 34, 'main': 155, 'method': 162, 'console': 42, 'run': 218, 'enter': 81, 'view': 279, 'correctly': 49, 'bug': 24, 'changes': 32, 'users': 276, 'files': 95, 'views': 281, 'resource': 211, 'doesnt': 71, 'text': 254, 'editor': 78, 'try': 266, 'does': 70, 'using': 277, 'ok': 176, 'location': 149, 'opening': 179, 'launch': 138, 'resources': 212, 'html': 109, 'page': 186, 'look': 152, 'display': 67, 'missing': 163, 'just': 134, 'button': 26, 'd

In [30]:
print(tfidf.fit(words))

TfidfVectorizer(max_features=300, norm='l1', stop_words='english')


In [29]:
duplicate_2.shape

(122075, 300)

In [25]:
x = abs(duplicate_1 - duplicate_2)
y = data['Label']

In [26]:
x.shape

(122075, 300)

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D


model = Sequential()
model.add(Dense(64,input_shape=(300,)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                19264     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6

In [14]:
results = model.fit(x_train.todense(),y_train,batch_size=64,epochs=5,verbose=1,validation_data=(x_test.todense(),y_test))

Epoch 1/5
1526/1526 [==============================] - 13s 7ms/step - loss: 0.4990 - acc: 0.7579 - val_loss: 0.3439 - val_acc: 0.8637
Epoch 2/5
1526/1526 [==============================] - 7s 4ms/step - loss: 0.3491 - acc: 0.8607 - val_loss: 0.3325 - val_acc: 0.8668
Epoch 3/5
1526/1526 [==============================] - 7s 5ms/step - loss: 0.3349 - acc: 0.8662 - val_loss: 0.3238 - val_acc: 0.8700
Epoch 4/5
1526/1526 [==============================] - 7s 5ms/step - loss: 0.3239 - acc: 0.8722 - val_loss: 0.3192 - val_acc: 0.8725
Epoch 5/5
1526/1526 [==============================] - 10s 6ms/step - loss: 0.3121 - acc: 0.8769 - val_loss: 0.3198 - val_acc: 0.8719


In [15]:
from sklearn.metrics import classification_report
pred = model.predict_classes(x_test)
print(classification_report(y_test,pred,labels=[1,0]))

/Users/rochelleallan/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           1       0.90      0.71      0.79      8294
           0       0.86      0.96      0.91     16121

    accuracy                           0.87     24415
   macro avg       0.88      0.83      0.85     24415
weighted avg       0.87      0.87      0.87     24415



In [19]:
x_test.shape

(24415, 300)

In [20]:
x_train.shape

(97660, 300)

In [21]:
y_test.shape

(24415,)

In [22]:
y_train.shape

(97660,)